# Play with MOABB on the Thielen dataset

## Do the import

In [2]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import mne
from mne.decoding import CSP
from mne.decoding import Vectorizer
from pyriemann.estimation import Xdawn, XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.classification import MDM
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC


import moabb


To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\moabb\pipelines\__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


In [3]:
from moabb.datasets import Thielen2021
from moabb.evaluations import WithinSessionEvaluation
from moabb.paradigms import CVEP

moabb.set_log_level("info")
warnings.filterwarnings("ignore")


## Instantiating Dataset
 ---------------------

 The first thing to do is to instantiate the dataset that we want to analyze.
 MOABB has a list of many different datasets, each one containing all the
 necessary information for describing them, such as the number of subjects,
 size of trials, names of classes, etc.

 The dataset class has methods for:

 - downloading its files from some online source (e.g. Zenodo)
 - importing the data from the files in whatever extension they might be
   (like .mat, .gdf, etc.) and instantiate a Raw object from the MNE package

In [4]:
datasets = Thielen2021()
print(datasets.subject_list)
print(datasets.event_id)

datasets.subject_list = [1, 2, 3]


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
{'1.0': 101, '0.0': 100}


In [5]:
sessions = datasets.get_data(subjects=[1])

print(sessions)

subject = 1
session_name = "0"
run_name = "0"
raw = sessions[subject][session_name][run_name]
raw.annotations.delete(np.linspace(0,37800-1,37800,dtype=int))


print(raw)


{1: {'0': {'0': <RawGDF | sub-01_20181128_block_1_main_eeg.gdf, 11 x 341479 (666.9 s), ~28.7 MB, data loaded>, '1': <RawGDF | sub-01_20181128_block_2_main_eeg.gdf, 11 x 341107 (666.2 s), ~28.6 MB, data loaded>, '2': <RawGDF | sub-01_20181128_block_3_main_eeg.gdf, 11 x 341105 (666.2 s), ~28.6 MB, data loaded>, '3': <RawGDF | sub-01_20181128_block_4_main_eeg.gdf, 11 x 341018 (666.0 s), ~28.6 MB, data loaded>, '4': <RawGDF | sub-01_20181128_block_5_main_eeg.gdf, 11 x 340901 (665.8 s), ~28.6 MB, data loaded>}}}
<RawGDF | sub-01_20181128_block_1_main_eeg.gdf, 11 x 341479 (666.9 s), ~28.7 MB, data loaded>


In [6]:
events = mne.find_events(raw)
print(events)

cond = np.sum(events[:,2]==200)
print(cond)
print(events[np.where(cond)])

epochs = mne.Epochs(raw, events[np.where(cond)], tmin=-0.3, tmax=0.7)
print(epochs)


[[  5946      0    100]
 [  5946      0    257]
 [  5946      0    204]
 ...
 [341085      0    100]
 [341093      0    101]
 [341104      0    257]]
1
[[5946    0  100]]
<Epochs |  1 events (good & bad), -0.300781 – 0.699219 s, baseline -0.300781 – 0 s, ~18 kB, data not loaded,
 '100': 1>


In [7]:
%matplotlib qt

In [8]:

# print(len(raw[1][0][0]))
# eeg = raw[8][0][0]
# plt.plot(np.linspace(0,len(eeg)*1/512,len(eeg)), eeg)
# plt.show()

rawp = raw.copy()
rawp.events = events[:1, :]
# rawp.plot_sensors()
# rawp.plot_psd()
print(rawp)
# rawp.plot(events=None)
# epochs.plot(n_epochs=1, events=True)

<RawGDF | sub-01_20181128_block_1_main_eeg.gdf, 11 x 341479 (666.9 s), ~28.7 MB, data loaded>


## Choosing a Paradigm

In [9]:
paradigm=CVEP()
print(paradigm==None)

2023-11-29 15:42:28,212 WARNING MainThread moabb.paradigms.cvep Choosing the first None classes from all possible events.


False


In [10]:
#X, labels, meta = paradigm.get_data(dataset=datasets, subjects=datasets.subject_list)
print(datasets.event_id)

{'1.0': 101, '0.0': 100}


In [11]:

pipelines = {}
pipelines["RG+LDA"]=make_pipeline(
    XdawnCovariances(
        nfilter=2, estimator="lwf", xdawn_estimator="scm"
    ),
    TangentSpace(),
    LDA(solver="lsqr", shrinkage="auto"),
)

pipelines["Xdw+LDA"] = make_pipeline(
    Xdawn(nfilter=2, estimator="scm"), Vectorizer(), LDA(solver="lsqr", shrinkage="auto")
)
pipelines["RG+SVC"]=make_pipeline(
    XdawnCovariances(
        nfilter=2, estimator="lwf", xdawn_estimator="scm"
    ),
    MDM()
)


In [12]:
paradigm = CVEP(resample=128)
print("charging dataset")
print(datasets.event_id)
print(events)
datasets.subject_list = datasets.subject_list[:]
print(datasets.subject_list)
datasets = [datasets]
overwrite = True  # set to True if we want to overwrite cached results
evaluation = WithinSessionEvaluation(
    paradigm=paradigm, datasets=datasets, suffix="examples", overwrite=overwrite
)

results = evaluation.process(pipelines)

2023-11-29 15:42:28,227 WARNING MainThread moabb.paradigms.cvep Choosing the first None classes from all possible events.
2023-11-29 15:42:28,229 INFO MainThread moabb.evaluations.base Processing dataset: Thielen2021


charging dataset
{'1.0': 101, '0.0': 100}
[[  5946      0    100]
 [  5946      0    257]
 [  5946      0    204]
 ...
 [341085      0    100]
 [341093      0    101]
 [341104      0    257]]
[1, 2, 3]


Thielen2021-WithinSession:   0%|          | 0/3 [00:00<?, ?it/s]

No hdf5_path provided, models will not be saved.


2023-11-29 15:46:48,983 INFO MainThread moabb.evaluations.base RG+LDA | Thielen2021 | 1 | 0: Score 0.645


...

2023-11-29 16:02:02,578 INFO MainThread moabb.evaluations.base RG+LDA | Thielen2021 | 3 | 0: Score 0.619


No hdf5_path provided, models will not be saved.


2023-11-29 16:02:09,214 INFO MainThread moabb.evaluations.base Xdw+LDA | Thielen2021 | 3 | 0: Score 0.639


No hdf5_path provided, models will not be saved.


2023-11-29 16:05:35,612 INFO MainThread moabb.evaluations.base RG+SVC | Thielen2021 | 3 | 0: Score 0.593
Thielen2021-WithinSession: 100%|██████████| 3/3 [23:07<00:00, 462.49s/it]


In [13]:
print(results)

      score       time   samples subject session  channels  n_sessions  \
0  0.684291   1.233204  189000.0       1       0         8           1   
1  0.587513   1.242908  189000.0       2       0         8           1   
2  0.638672   1.310175  189000.0       3       0         8           1   
3  0.645069  39.414417  189000.0       1       0         8           1   
4  0.575317  37.285667  189000.0       2       0         8           1   
5  0.619106  38.912666  189000.0       3       0         8           1   
6  0.624101  40.339073  189000.0       1       0         8           1   
7  0.551016  38.957550  189000.0       2       0         8           1   
8  0.592974  41.263187  189000.0       3       0         8           1   

       dataset pipeline  
0  Thielen2021  Xdw+LDA  
1  Thielen2021  Xdw+LDA  
2  Thielen2021  Xdw+LDA  
3  Thielen2021   RG+LDA  
4  Thielen2021   RG+LDA  
5  Thielen2021   RG+LDA  
6  Thielen2021   RG+SVC  
7  Thielen2021   RG+SVC  
8  Thielen2021   RG+SVC  

# PLot Result

In [14]:
fig, ax = plt.subplots(facecolor="white", figsize=[8, 4])

sns.stripplot(
    data=results,
    y="score",
    x="pipeline",
    ax=ax,
    jitter=True,
    alpha=0.5,
    zorder=1,
    palette="Set1",
)
sns.pointplot(data=results, y="score", x="pipeline", ax=ax, palette="Set1")

ax.set_ylabel("ROC AUC")
ax.set_ylim(0.5, 1)

plt.show()